In [3]:
import os
from keras.models import load_model
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout,Convolution2D,MaxPooling2D,Flatten
from keras.optimizers import Adam


# number of classes
nb_classes = 43

# input image dimensions
img_rows, img_cols, img_ch = 32, 32, 3

# number of convolutional filters to use
nb_filter1 = 32
nb_filter2 = 64
nb_filter3 = 128



model = Sequential()
model.add(Convolution2D(32, kernel_size=(5, 5),padding='same',
                        input_shape=(img_rows,img_cols,img_ch),name='conv1'))
model.add(Activation('relu',name='relu1'))
model.add(MaxPooling2D(pool_size=(2,2),strides=1,name='maxpool1'))
model.add(Convolution2D(64, kernel_size=(5, 5),padding='same',
                        name='conv2'))
model.add(Activation('relu',name='relu2'))
model.add(MaxPooling2D(pool_size=(2,2),strides=None,name='maxpool2'))
model.add(Convolution2D(128, kernel_size=(5, 5), padding='same',
                        name='conv3'))
model.add(Activation('relu',name='relu3'))
model.add(MaxPooling2D(pool_size=(2,2),strides=None,name='maxpool3'))
model.add(Flatten(name='flatten'))
model.add(Dropout(0.5,name='dropout1'))
model.add(Dense(128, name='hidden1'))
model.add(Activation('relu',name='relu4'))
model.add(Dropout(0.5,name='dropout2'))
model.add(Dense(128,  name='hidden2'))
model.add(Activation('relu',name='relu5'))
model.add(Dense(43, name='output'))
model.add(Activation('softmax',name='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv2D)               (None, 32, 32, 384)       29184     
_________________________________________________________________
relu1 (Activation)           (None, 32, 32, 384)       0         
_________________________________________________________________
maxpool1 (MaxPooling2D)      (None, 31, 31, 384)       0         
_________________________________________________________________
conv2 (Conv2D)               (None, 31, 31, 64)        614464    
_________________________________________________________________
relu2 (Activation)           (None, 31, 31, 64)        0         
_________________________________________________________________
maxpool2 (MaxPooling2D)      (None, 15, 15, 64)        0         
_________________________________________________________________
conv3 (Conv2D)               (None, 15, 15, 128)       204928    
__________

In [10]:
import pickle

training_file = 'train.p'
testing_file = 'test.p'
valid_file = 'valid.p'
 
with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
with open(valid_file, mode='rb') as f:
    valid = pickle.load(f)

train

{'coords': array([[  6,   5,  21,  20],
        [  6,   6,  22,  22],
        [  5,   6,  22,  23],
        ..., 
        [ 17,  15, 178, 155],
        [ 17,  15, 183, 160],
        [ 20,  18, 211, 184]], dtype=uint8),
 'features': array([[[[ 28,  25,  24],
          [ 27,  24,  23],
          [ 27,  24,  22],
          ..., 
          [ 32,  28,  24],
          [ 31,  27,  25],
          [ 31,  27,  26]],
 
         [[ 29,  26,  25],
          [ 27,  25,  23],
          [ 27,  25,  23],
          ..., 
          [ 32,  28,  24],
          [ 31,  27,  24],
          [ 30,  27,  25]],
 
         [[ 28,  26,  26],
          [ 27,  25,  23],
          [ 26,  25,  23],
          ..., 
          [ 32,  28,  24],
          [ 31,  27,  24],
          [ 30,  27,  25]],
 
         ..., 
         [[ 27,  24,  23],
          [ 28,  25,  24],
          [ 30,  25,  24],
          ..., 
          [ 27,  24,  23],
          [ 28,  24,  22],
          [ 29,  25,  22]],
 
         [[ 28,  23,  23],
   

In [6]:
    X_train, y_train, X_test, y_test = train['features'], train['labels'],test['features'], test['labels']
    X_valid, y_valid = test['features'], test['labels']

In [8]:
y_train[0]

41

In [13]:
from sklearn.preprocessing import LabelEncoder
import keras
from keras.utils.np_utils import to_categorical
import numpy as np

def one_hot(Y):
    encoder = LabelEncoder()
    encoder.fit(Y)
    encoded_Y = encoder.transform(Y)
    return to_categorical(encoded_Y)
   

y_train = one_hot(y_train)
y_test  = one_hot(y_test)
y_valid = one_hot(y_valid)

def normalize(X_train):
    return np.divide(np.subtract(X_train,128.),128.)


def normalize_images(X):
    return normalize(X)

X_train = normalize_images(X_train)
X_test = normalize_images(X_test)
X_valid = normalize_images(X_valid)


In [14]:
import time

nb_epoch = 1000
batch_size=128

model_name='test.chkpt'

if os.path.isfile(model_name):
    try:
        model = load_model(model_name)
        print('loading trained model ...')
    except Exception as e:
        print('Unable to load model', model_name, ':', e)
        raise    


adam = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(optimizer=adam,
          loss='categorical_crossentropy',
          metrics=['accuracy'])

startTime = time.time()
history = model.fit(X_train, y_train,
                    batch_size=batch_size, epochs=nb_epoch,
                    verbose=2, validation_data=(X_valid, y_valid))

endTime = time.time()
print('elapsed secs:', (endTime - startTime)/1000.)

Train on 34799 samples, validate on 12630 samples
Epoch 1/1000


KeyboardInterrupt: 

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")